In [1]:
# !pip install -U transformers accelerate datasets bertviz
!pip install -q -U datasets

In [2]:
import pprint

from datasets import load_dataset


from transformers import BartForQuestionAnswering   # not using

from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import TrainingArguments, Trainer

from transformers import T5Tokenizer, TFT5Model, TFT5ForConditionalGeneration
import tensorflow as tf

import torch

In [45]:
!pip install -q evaluate
import evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00


In [46]:
!pip install -q rouge_score

  Preparing metadata (setup.py) ... done


# Import Data

In [4]:
ds = load_dataset("WorkInTheDark/FairytaleQA")

pprint.pprint(ds['train'][1])

RuntimeError: Dataset scripts are no longer supported, but found FairytaleQA.py

In [ ]:
# use one toy example

toy_ex=ds['train'][1]
context=toy_ex['story_section']
question=toy_ex['question']
answer=toy_ex['answer1']


pprint.pprint(f"CONTEXT: {context}")
pprint.pprint(f"QUESTION: {question}")
pprint.pprint(f"ANSWER: {answer}")



('CONTEXT: once upon a time there was a king who went forth into the world and '
 'fetched back a beautiful queen . and after they had been married a while god '
 'gave them a little daughter . then there was great rejoicing in the city and '
 'throughout the country , for the people wished their king all that was good '
 ', since he was kind and just . while the child lay in its cradle , a strange '
 '- looking old woman entered the room , and no one knew who she was nor '
 'whence she came . the old woman spoke a verse over the child , and said that '
 'she must not be allowed out under the open sky until she were full fifteen '
 'years of age , since otherwise the mountain troll would fetch her . when the '
 'king heard this he took her words to heart , and posted guards to watch over '
 'the little princess so that she would not get out under the open sky .')
'QUESTION: what type of ruler was the king ?'
'ANSWER: kind and just .'


In [82]:
# use one toy example

toy_ex=ds['train'][2]
context=toy_ex['story_section']
question=toy_ex['question']
answer=toy_ex['answer1']


pprint.pprint(f"CONTEXT: {context}")
pprint.pprint(f"QUESTION: {question}")
pprint.pprint(f"ANSWER: {answer}")



('CONTEXT: once upon a time there was a king who went forth into the world and '
 'fetched back a beautiful queen . and after they had been married a while god '
 'gave them a little daughter . then there was great rejoicing in the city and '
 'throughout the country , for the people wished their king all that was good '
 ', since he was kind and just . while the child lay in its cradle , a strange '
 '- looking old woman entered the room , and no one knew who she was nor '
 'whence she came . the old woman spoke a verse over the child , and said that '
 'she must not be allowed out under the open sky until she were full fifteen '
 'years of age , since otherwise the mountain troll would fetch her . when the '
 'king heard this he took her words to heart , and posted guards to watch over '
 'the little princess so that she would not get out under the open sky .')
'QUESTION: who entered the room while the child lay in its cradle ?'
'ANSWER: a strange - looking old woman .'


# Testing Models

BartForQuestionAnswering is for position based extraction?

### Bart Base

In [83]:
# bartmodel = BartForQuestionAnswering.from_pretrained('facebook/bart-base')

bartmodel = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
barttokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

input_text = f"question: {question} context: {context}"
inputs = barttokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)


summary_ids = bartmodel.generate(inputs['input_ids'], num_beams=4, max_length=512, early_stopping=True)
generated_answer = [barttokenizer.decode(g, skip_special_tokens=True) for g in summary_ids]



In [84]:
generated_answer

['question: who entered the room while the child lay in its cradle ? context: once upon a time there was a king who went forth into the world and fetched back a beautiful queen . and after they had been married a while god gave them a little daughter . then there was great rejoicing in the city and throughout the country , for the people wished their king all that was good , since he was kind and just . while the girl lay in her cradle , a strange - looking old woman entered the Room , and no one knew who she was nor whence she came . the old woman spoke a verse over the child , and said that she must not be allowed out under the open sky until she were full fifteen years of age , since otherwise the mountain troll would fetch her . when the king heard this he took her words to heart , and posted guards to watch over the little princess so that she would not get out under-the open sky .']

In [85]:
pprint.pprint(input_text)
pprint.pprint(f"\nA*** {generated_answer}")

rouge = evaluate.load('rouge')
predictions = [generated_answer[0]]
references = [answer]
rouge_scores = rouge.compute(predictions=predictions, references=references)
print(rouge_scores)

('question: who entered the room while the child lay in its cradle ? context: '
 'once upon a time there was a king who went forth into the world and fetched '
 'back a beautiful queen . and after they had been married a while god gave '
 'them a little daughter . then there was great rejoicing in the city and '
 'throughout the country , for the people wished their king all that was good '
 ', since he was kind and just . while the child lay in its cradle , a strange '
 '- looking old woman entered the room , and no one knew who she was nor '
 'whence she came . the old woman spoke a verse over the child , and said that '
 'she must not be allowed out under the open sky until she were full fifteen '
 'years of age , since otherwise the mountain troll would fetch her . when the '
 'king heard this he took her words to heart , and posted guards to watch over '
 'the little princess so that she would not get out under the open sky .')
('\n'
 "A*** ['question: who entered the room while t

### T5 Base

In [ ]:
t5model = TFT5ForConditionalGeneration.from_pretrained('t5-base') #also t5-small and t5-large
t5tokenizer = T5Tokenizer.from_pretrained('t5-base')

input_text = f"question: {question} context: {context}"
inputs = t5tokenizer(input_text, return_tensors="tf")

t5_summary_ids = t5model.generate(inputs['input_ids'], num_beams=3, no_repeat_ngram_size=3)
t5generatedanswer= [t5tokenizer.decode(g, skip_special_tokens=True,clean_up_tokenization_spaces=False) for g in t5_summary_ids]


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
/usr/local/lib/python3.11/dist-packages/transformers/generation/tf_utils.py:836: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [63]:
pprint.pprint(input_text)
pprint.pprint(f"\nA*** {t5generatedanswer}")

rouge = evaluate.load('rouge')
predictions = [t5generatedanswer[0]]
references = [answer]
rouge_scores = rouge.compute(predictions=predictions, references=references)
print(rouge_scores)

('question: what type of ruler was the king ? context: once upon a time there '
 'was a king who went forth into the world and fetched back a beautiful queen '
 '. and after they had been married a while god gave them a little daughter . '
 'then there was great rejoicing in the city and throughout the country , for '
 'the people wished their king all that was good , since he was kind and just '
 '. while the child lay in its cradle , a strange - looking old woman entered '
 'the room , and no one knew who she was nor whence she came . the old woman '
 'spoke a verse over the child , and said that she must not be allowed out '
 'under the open sky until she were full fifteen years of age , since '
 'otherwise the mountain troll would fetch her . when the king heard this he '
 'took her words to heart , and posted guards to watch over the little '
 'princess so that she would not get out under the open sky .')
"\nA*** ['kind and just']"
{'rouge1': np.float64(1.0), 'rouge2': np.float64(